# How to change the plotting backend

optimagic supports various visualization libraries as plotting backends, which can be
selected using the `backend` argument.

::::{tab-set}

:::{tab-item} Plotly

The default plotting library. To select the Plotly backend explicitly, set `backend="plotly"`.

The returned figure object is a [`plotly.graph_objects.Figure`](https://plotly.com/python-api-reference/generated/plotly.graph_objects.Figure.html).

:::

:::{tab-item} Matplotlib

To select the Matplotlib backend, set `backend="matplotlib"`.

The returned figure object is a [`matplotlib.axes.Axes`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.html).

```{note}
In case of grid plots (such as `convergence_plot` or `slice_plot`), the returned object is a 2-dimensional numpy array of `Axes` objects: [`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html)[[`matplotlib.axes.Axes`]](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.html) of shape `(n_rows, n_cols)`.
```

:::

:::{tab-item} Bokeh

To select the Bokeh backend, set `backend="bokeh"`.

The returned figure object is a [`bokeh.plotting.figure`](https://docs.bokeh.org/en/latest/docs/reference/plotting/figure.html).

```{note}
In case of grid plots (such as `convergence_plot` or `slice_plot`), the returned object is a [`bokeh.models.GridPlot`](https://docs.bokeh.org/en/latest/docs/reference/models/plots.html#bokeh.models.GridPlot) object.
```

```{warning}
Bokeh applies themes globally. Passing the `template` parameter to a plotting function updates the theme for all existing and future Bokeh plots. If you do not pass `template`, a default template is applied, which will also change the global theme.
```
::::

In the following guide, we showcase the criterion plot visualized using different
backends.

In [ ]:
import numpy as np

import optimagic as om


def sphere(x):
    return x @ x


results = {}
for algo in ["scipy_lbfgsb", "scipy_neldermead"]:
    results[algo] = om.minimize(sphere, params=np.arange(5), algorithm=algo)

## Plotly

:::{note}

**Choose the Plotly renderer according to your environment:**

- Use `plotly.io.renderers.default = "notebook_connected"` in Jupyter notebooks for interactive plots.
- Use `plotly.io.renderers.default = "browser"` to open plots in your default web browser when running as a script.

Refer to the [Plotly documentation](https://plotly.com/python/renderers/) for more details.

:::

In [ ]:
import plotly.io as pio

pio.renderers.default = "notebook_connected"

fig = om.criterion_plot(results, backend="plotly")  # Also the default
fig.show()

## Matplotlib

In [ ]:
ax = om.criterion_plot(results, backend="matplotlib")

## Bokeh

In [ ]:
from bokeh.io import output_notebook, show

output_notebook()

p = om.criterion_plot(results, backend="bokeh")
show(p)